# Deploy model and run inference

In [1]:
%%capture
%pip install sagemaker -U

In [2]:
import os
import glob
import cv2
import sagemaker
from sagemaker.tensorflow import TensorFlowModel
from sagemaker.utils import name_from_base
%matplotlib inline
import matplotlib.pyplot as plt
import visualization_utils as viz_utils

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:172: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[12/01/24 22:58:56] INFO     Found credentials from IAM Role:                                   ]8;id=202883;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=548401;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


2024-12-01 22:58:57.303858: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733093937.323894   31633 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733093937.329925   31633 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-01 22:58:57.349770: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
role = sagemaker.get_execution_role()

[12/01/24 22:58:58] INFO     Found credentials from IAM Role:                                   ]8;id=885103;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=240544;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

Now that you have trained successfully your model, you want to look at the predictions on some sample images. To do so, you will need to find the s3 path of the exported model. You can navigate to the Training jobs section of the AWS web UI and click on the training job of interest. Scroll down and you should see something like this:

![Example Artifact](../data/example_artifact.png)

The model artifact path should look something like 
```s3://sagemaker-us-east-1-073338978050/tf2-object-detection-2022-10-22-21-26-37-033/output/model.tar.gz```. Use this value to update the `model_artifact` variable below.

In [4]:
# TODO: Update the model artifact here. 
model_artifact = 's3://sagemaker-us-east-1-470901650325/tf2-object-detection-2024-12-01-15-41-05-719/output/model.tar.gz'

Now we can deploy the model. Run the following cell and check that the model was correctly deployed by navigating to Inference endpoints in the web UI.

![Example endpoint](../data/example_endpoints.png)


In [5]:
model = TensorFlowModel(
    name=name_from_base('tf2-object-detection'),
    model_data=model_artifact,
    role=role,
    framework_version='2.8'
)

predictor = model.deploy(initial_instance_count=1, instance_type='ml.g4dn.xlarge')

[12/01/24 22:59:01] INFO     Found credentials from IAM Role:                                   ]8;id=523989;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=202823;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     image_uri is not presented, retrieving image_uri based on instance_type,  ]8;id=667112;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/tensorflow/model.py\model.py]8;;\:]8;id=563660;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/tensorflow/model.py#508\508]8;;\
                             framework etc.                                                                        

                    INFO     Creating model with name: tf2-object-detection-2024-12-01-22-59-01-014 ]8;id=893661;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=621792;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4025\4025]8;;\

[12/01/24 22:59:02] INFO     Creating endpoint-config with name                                     ]8;id=158863;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=901668;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#5820\5820]8;;\
                             tf2-object-detection-2024-12-01-22-59-02-272                                          

                    INFO     Creating endpoint with name                                            ]8;id=270591;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=917957;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4642\4642]8;;\
                             tf2-object-detection-2024-12-01-22-59-02-272                                          

                    ERROR    Please check the troubleshooting guide for common errors:              ]8;id=213798;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=834579;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4665\4665]8;;\
                             https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-python-sdk-t                
                             roubleshooting.html#sagemaker-python-sdk-troubleshooting-create-endpoi                
                             nt                                                                                    

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:8                                                                                    │
│                                                                                                  │
│   5 │   framework_version='2.8'                                                                  │
│   6 )                                                                                            │
│   7                                                                                              │
│ ❱ 8 predictor = model.deploy(initial_instance_count=1, instance_type='ml.g4dn.xlarge')           │
│   9                                                                                              │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/tensorflow/model.py │
│ :368 in deploy                                                                                   │
│                                                                                                  │
│   365 │   │   │   msg = "The TensorFlow version %s doesn't support EIA." % self.framework_vers   │
│   366 │   │   │   raise AttributeError(msg)                                                      │
│   367 │   │                                                                                      │
│ ❱ 368 │   │   return super(TensorFlowModel, self).deploy(                                        │
│   369 │   │   │   initial_instance_count=initial_instance_count,                                 │
│   370 │   │   │   instance_type=instance_type,                                                   │
│   371 │   │   │   serializer=serializer,                                                         │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/model.py:1778 in    │
│ deploy                                                                                           │
│                                                                                                  │
│   1775 │   │   │   if is_explainer_enabled:                                                      │
│   1776 │   │   │   │   explainer_config_dict = explainer_config._to_request_dict()               │
│   1777 │   │   │                                                                                 │
│ ❱ 1778 │   │   │   self.sagemaker_session.endpoint_from_production_variants(                     │
│   1779 │   │   │   │   name=self.endpoint_name,                                                  │
│   1780 │   │   │   │   production_variants=[production_variant],                                 │
│   1781 │   │   │   │   tags=tags,                                                                │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py:5823 in  │
│ endpoint_from_production_variants                                                                │
│                                                                                                  │
│   5820 │   │   logger.info("Creating endpoint-config with name %s", name)                        │
│   5821 │   │   self.sagemaker_client.create_endpoint_config(**config_options)                    │
│   5822 │   │                                                                                     │
│ ❱ 5823 │   │   return self.create_endpoint(                                                      │
│   5824 │   │   │   endpoint_name=name,                                                           │
│   5825 │   │   │   config_name=name,                       

## Run inference

Our model is now deployed and we can query it. We are going to use the images available in `data/test_video` to run inference and generate a video. To do so, we are going to need a few tools:
* we need to sort all the frames by index order (which corresponds to chronological order)
* we need a function to load images into numpy array
* we need a loop to run inference and display the results on the input image

We list the frame paths and sort them by index.

In [ ]:
frames_path = sorted(glob.glob('../data/test_video/*.png'), 
                     key = lambda k: int(os.path.basename(k).split('.')[0].split('_')[1]))

We create a small function to load images.

In [ ]:
import numpy as np
def load_image(path: str) -> np.ndarray:
    """Read an image from the path and returns a numpy array"""
    cv_img = cv2.imread(path,1).astype('uint8')
    cv_img = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
    return cv_img

We create a mapping from id to name for visualization purposes.

In [ ]:
category_index = {
                    1: {'id': 1, 'name': 'vehicle'}, 
                    2: {'id': 2, 'name': 'pedestrian'},
                    4: {'id': 4, 'name': 'cyclist'}
                }

This is the main loop:
* we load images to numpy
* we query the deployed model
* we display the inference results on the images

In [ ]:
def image_file_to_tensor(path):
    cv_img = cv2.imread(path,1).astype('uint8')
    cv_img = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
    return cv_img

images = []
for idx, path in enumerate(frames_path):
    if idx % 10 == 0:
        print(f'Processed {idx}/{len(frames_path)} images.')
        
    # load image
    img = image_file_to_tensor(path)
    inputs = {'instances': [img.tolist()]}
    
    # run inference and extract results
    detections = predictor.predict(inputs)['predictions'][0]
    detection_boxes = np.array(detections['detection_boxes'])
    detection_classes = [int(x) for x in detections['detection_classes']]
    detection_scores = detections['detection_scores']
    
    # display results on image
    image_np_with_detections = \
        viz_utils.visualize_boxes_and_labels_on_image_array(
            img,
            detection_boxes,
            detection_classes,
            detection_scores,
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=100,
            min_score_thresh=0.6,
            agnostic_mode=False)
    images.append(image_np_with_detections)

We can verify that the model worked correctly by displaying elements of the `images` list.

In [ ]:
plt.imshow(images[0])

Finally, we can create a video (`output.avi`) with our detections by running the following function.

In [ ]:
frame_width = images[0].shape[0]
frame_height = images[0].shape[1]

out = cv2.VideoWriter('output.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))

# Read and display the images
for image in images:
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    out.write(image) # Write the image to the video
    if cv2.waitKey(1) == ord('q'): # Hit `q` to exit
        break
        
# Release everything if job is finished
out.release()
cv2.destroyAllWindows()

The video would be stored in the current working directory. You can download it from Sagemaker and run it locally.